# Análise Exploratória no Ministério da Economia 

> Este projeto de Análise Exploratória de Dados tem como objetivo responder 05 perguntas de negócios sobre as Importações Brasileiras no primeiro semestre de 2021 e 2022, como fonte de estudos, utilizo dados extraídos do <a href="https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta">Ministério da Economia.</a>

Após a conclusão deste projeto, podemos responder as seguintes perguntas de negócios:
- 01 - Qual é o valor total das Importações no período?
- 02 - Qual foi o custo das Importações por Estado?
- 03 - Qual foi o custo das Importações por Municípios?
- 04 - Quais são os principais Remetentes das Importações?
- 05 - Quais são os Produtos com Maiores Importações?

### Importando Bibliotecas e Pacotes

In [84]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn.feature_extraction.text import CountVectorizer

### Acessando os dados (ref.2021)

In [2]:
#(0) Faço a importação do arquivo csv a ser analisado, (1) crio um DataFrame(df) renomeando as colunas já existentes (2)e visualizo o início.
imp_21 = "IMP_2021_MUN.csv"
dadosi21 = pd.read_csv(imp_21, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)

/tmp/ipykernel_359309/1956096061.py:3: DtypeWarning: Columns (0,1,2,3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dadosi21 = pd.read_csv(imp_21, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)


### Acessando os dados (ref.2022)

In [3]:
#(0) Faço a importação do arquivo csv a ser analisado, (1) crio um DataFrame(df) renomeando as colunas já existentes (2)e visualizo o início.
imp_22 = "IMP_2022_MUN.csv"
dadosi22 = pd.read_csv(imp_22, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)

/tmp/ipykernel_359309/1510667406.py:3: DtypeWarning: Columns (0,1,2,3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dadosi22 = pd.read_csv(imp_22, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)


### Acessando as nomeclaturas originais

In [4]:
# Realizando a leitura dos arquivos já convertidos e carregando em uma variável
with open("meses.json", "r") as jsonMes:
    meses = json.loads(jsonMes.read())
with open("produtos.json", "r") as jsonProdutos:
    produtos = json.loads(jsonProdutos.read())
with open("paises.json", "r") as jsonPais:
    paises = json.loads(jsonPais.read())
with open("estadosBr.json", "r") as jsonEstados:
    estados = json.loads(jsonEstados.read())
with open("municipios.json", "r") as jsonMun:
    municipios = json.loads(jsonMun.read())

### Limpeza e Tratamento dos Dados (ref.2021)

In [5]:
# Altero o tipo de dados das colunas para utilização com o map
dadosi21["Mês"] = dadosi21["Mês"].astype('string')
dadosi21["Produtos"] = dadosi21["Produtos"].astype('string')
dadosi21["País(orig)"] = dadosi21["País(orig)"].astype('string')
dadosi21["Estado(dest)"] = dadosi21["Estado(dest)"].astype('string')
dadosi21["Município(dest)"] = dadosi21["Município(dest)"].astype('string')

In [6]:
# Com a função drop removo a coluna "ano" - e a linha "0".
dadosi21.drop(["Ano"], axis=1, inplace=True)
dadosi21.drop([0], inplace=True)

# Por fim faço a remoção de duplicatas e valores NA
dadosi21 = dadosi21.drop_duplicates()
dadosi21 = dadosi21.dropna()

In [7]:
# Renomeio todas as colunas abaixo utilizando como valor de saída as Nomeclaturas originais
dadosi21["Mês"] = dadosi21["Mês"].map(meses,na_action='ignore')
dadosi21["Produtos"] = dadosi21["Produtos"].map(produtos,na_action='ignore')
dadosi21["País(orig)"] = dadosi21["País(orig)"].map(paises,na_action='ignore')
dadosi21["Estado(dest)"] = dadosi21["Estado(dest)"].map(estados,na_action='ignore')
dadosi21["Município(dest)"] = dadosi21["Município(dest)"].map(municipios,na_action='ignore')

In [8]:
# Seleciono o período a ser analisado com a função loc (Jan,Fev,Mar,Abril,Maio,Junho) 
dadosi21 = dadosi21.loc[dadosi21['Mês'].isin(['Janeiro','Fevereiro','Março',
                                              'Abril','Maio','Junho'])]

In [9]:
# Incluo o 'index' do pandas como propriedade do DF e reordeno o DF
dadosi21["Id_Compra"] = dadosi21.index
dadosi21 = dadosi21[['Id_Compra','Município(dest)','Estado(dest)','País(orig)','Produtos',
                     'Total_em_Dólar','KG_Liq','Mês']]

In [10]:
dadosi21

,Id_Compra,Município(dest),Estado(dest),País(orig),Produtos,Total_em_Dólar,KG_Liq,Mês
2,2,Jaguariúna,São Paulo,China,Aparelhos emissores (transmissores) para radio...,3685,1,Abril
4,4,Indaiatuba,São Paulo,Estados Unidos,"Tubos de borracha vulcanizada não endurecida, ...",207124,10417,Janeiro
7,7,Itajaí,Santa Catarina,China,"Tecidos de fios de filamentos sintéticos, incl...",5610763,2070874,Março
9,9,Embu,São Paulo,Polônia,"Tachas, pregos, percevejos e artefatos semelha...",14,0,Março
11,11,Ipatinga,Minas Gerais,China,"Poliacetais, outros poliéteres e resinas epóxi...",99836,44000,Abril
...,...,...,...,...,...,...,...,...
1556165,1556165,Mafra,Santa Catarina,China,"Adubos (fertilizantes) minerais ou químicos, c...",548565,2003425,Janeiro
1556166,1556166,Taubaté,São Paulo,Espanha,"Barras e perfis, de outras ligas de aço; barra...",535,14,Março
1556167,1556167,Barueri,São Paulo,Coreia do Sul,"Espelhos de vidro, mesmo emoldurados, incluído...",5555,116,Maio
1556168,1556168,Betim,Minas Gerais,China,"Outras formas (por exemplo: varetas, tubos, pe...",581,8,Maio


### Limpeza e Tratamento dos Dados (ref.2022)

In [11]:
# Altero o tipo de dados das colunas para utilização com o map
dadosi22["Mês"] = dadosi22["Mês"].astype('string')
dadosi22["Produtos"] = dadosi22["Produtos"].astype('string')
dadosi22["País(orig)"] = dadosi22["País(orig)"].astype('string')
dadosi22["Estado(dest)"] = dadosi22["Estado(dest)"].astype('string')
dadosi22["Município(dest)"] = dadosi22["Município(dest)"].astype('string')

In [12]:
# Com a função drop removo a coluna "ano" e a linha "0".
dadosi22.drop(["Ano"], axis=1, inplace=True)
dadosi22.drop([0], inplace=True)

# Por fim faço a remoção de duplicatas e valores NA
dadosi22 = dadosi22.drop_duplicates()
dadosi22 = dadosi22.dropna()

In [13]:
# Renomeio todas as colunas abaixo utilizando como valor de saída as Nomeclaturas originais
dadosi22["Mês"] = dadosi22["Mês"].map(meses,na_action='ignore')
dadosi22["Produtos"] = dadosi22["Produtos"].map(produtos,na_action='ignore')
dadosi22["País(orig)"] = dadosi22["País(orig)"].map(paises,na_action='ignore')
dadosi22["Estado(dest)"] = dadosi22["Estado(dest)"].map(estados,na_action='ignore')
dadosi22["Município(dest)"] = dadosi22["Município(dest)"].map(municipios,na_action='ignore')

In [14]:
# Seleciono o período a ser analisado com a função loc (Jan,Fev,Mar,Abril,Maio,Junho) 
dadosi22 = dadosi22.loc[dadosi22['Mês'].isin(['Janeiro','Fevereiro','Março',
                                              'Abril','Maio','Junho'])]

In [15]:
# Incluo o 'index' do pandas como propriedade do DF e reordeno o DF
dadosi22["Id_Compra"] = dadosi22.index
dadosi22 = dadosi22[['Id_Compra','Município(dest)','Estado(dest)','País(orig)','Produtos',
                     'Total_em_Dólar','KG_Liq','Mês']]

In [16]:
dadosi22

,Id_Compra,Município(dest),Estado(dest),País(orig),Produtos,Total_em_Dólar,KG_Liq,Mês
1,1,São Bernardo do Campo,São Paulo,"Tcheca, República",Outros motores e máquinas motrizes,26472,1553,Maio
2,2,Carlos Barbosa,Rio Grande do Sul,Alemanha,"Instrumentos, aparelhos e máquinas de medida o...",9418,350,Maio
3,3,São Paulo,São Paulo,Estados Unidos,"Bombas para líquidos, mesmo com dispositivo me...",316863,2547,Janeiro
4,4,Chapecó,Santa Catarina,Itália,"Refrigeradores, congeladores (freezers) e outr...",169432,11290,Março
5,5,Curitiba,Paraná,Suíça,"Parafusos, pernos ou pinos, roscados, porcas, ...",106388,2756,Janeiro
...,...,...,...,...,...,...,...,...
630790,630790,Guaratinguetá,São Paulo,China,Suportes preparados para gravação de som ou pa...,67,0,Abril
630791,630791,Osasco,São Paulo,Itália,Fornos elétricos industriais ou de laboratório...,14207,900,Abril
630792,630792,Mogi Guaçu,São Paulo,Itália,Partes reconhecíveis como exclusiva ou princip...,2006,17,Maio
630793,630793,Telêmaco Borba,Paraná,NaN,Partes reconhecíveis como exclusiva ou princip...,269,0,Abril


> # Iniciando a Análise Exploratória

### Qual é o valor total das Importações no período?

Informações sobre o custo das Importações (Comercio Exterior -> Brasil) e o total de operações.

In [17]:
# Altero o tipo das colunas que serão utilizadas
dadosi21['Total_em_Dólar'] = dadosi21['Total_em_Dólar'].astype('int')
dadosi22['Total_em_Dólar'] = dadosi22['Total_em_Dólar'].astype('int')

In [18]:
# Calculando os valores das importações de 2021
despesa_total21 = dadosi21["Total_em_Dólar"].sum()
despesa_count21 = dadosi21["Total_em_Dólar"].count()
despesa_media21 = dadosi21["Total_em_Dólar"].mean()

In [19]:
# Calculando os valores das importações 2022
despesa_total22 = dadosi22["Total_em_Dólar"].sum()
despesa_count22 = dadosi22["Total_em_Dólar"].count()
despesa_media22 = dadosi22["Total_em_Dólar"].mean()

In [20]:
# Crio um dataframe utilizando o "pandas"
despesas21e22 = pd.DataFrame({"Ano_Base": ["2021", "2022"],
                            "Total_de_Transacoes": (despesa_count21, despesa_count22),
                            "Total_Investido_em_Importacoes": (despesa_total21, despesa_total22),
                            "Valor_medio_das_Importacoes": (despesa_media21, despesa_media22)})
# Incluo uma coluna no meu DF com passando o 'Index'
despesas21e22["Despesas21e22_id"] = despesas21e22.index

In [21]:
# Edito a formatação numérica das colunas 
despesas21e22 ["Total_de_Transacoes"] = despesas21e22["Total_de_Transacoes"].map("{:,.0f}".format)
despesas21e22 ["Total_Investido_em_Importacoes"] = despesas21e22["Total_Investido_em_Importacoes"].map("${:,.0f}".format)
despesas21e22 ["Valor_medio_das_Importacoes"] = despesas21e22["Valor_medio_das_Importacoes"].map("${:,.0f}".format)

In [22]:
# Faço uma 'Reordenação' do DF
despesas21e22 = despesas21e22[['Despesas21e22_id','Ano_Base','Total_de_Transacoes',
                               'Total_Investido_em_Importacoes','Valor_medio_das_Importacoes']]

In [23]:
despesas21e22

,Despesas21e22_id,Ano_Base,Total_de_Transacoes,Total_Investido_em_Importacoes,Valor_medio_das_Importacoes
0,0,2021,"772,039","$99,176,454,736","$128,460"
1,1,2022,"630,794","$105,951,639,402","$167,966"


## Qual foi o custo das Importações por Estado?

Faço um resumo do custo das Importações por Estado, comparando os dois periódos.

### Ano 2021

In [24]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
uf_total21 = dadosi21.groupby('Estado(dest)')['Total_em_Dólar'].sum()
uf_media21 = dadosi21.groupby("Estado(dest)")["Total_em_Dólar"].mean()
uf_operacoes21 = dadosi21.groupby("Estado(dest)")["Total_em_Dólar"].count()

In [25]:
# Crio um DataFrame para armazenar os Valores agrupados
uf_analises21 = pd.DataFrame({"Numero_de_Transacoes" : uf_operacoes21,
                              "Valor_medio_das_Importacoes" : uf_media21,
                              "Valor_total_das_Importacoes" : uf_total21})

#  Incluo uma coluna no meu DF com passando o 'Index'
uf_analises21["Uf_analises21_id"] = uf_analises21.index

In [26]:
# Edito a formatação numérica dos dados
uf_analises21 = uf_analises21.round(2)
uf_analises21 ["Valor_medio_das_Importacoes"] = uf_analises21["Valor_medio_das_Importacoes"].map("${:,.2f}".format)
uf_analises21 ["Valor_total_das_Importacoes"] = uf_analises21["Valor_total_das_Importacoes"].map("${:,.2f}".format)

In [27]:
# Faço uma 'Reordenação' do DF
uf_analises21 = uf_analises21[['Uf_analises21_id','Numero_de_Transacoes','Valor_total_das_Importacoes',
                               'Valor_medio_das_Importacoes']]

### Ano 2022

In [28]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
uf_total22 = dadosi22.groupby('Estado(dest)')['Total_em_Dólar'].sum()
uf_media22 = dadosi22.groupby("Estado(dest)")["Total_em_Dólar"].mean()
uf_operacoes22 = dadosi22.groupby("Estado(dest)")["Total_em_Dólar"].count()

In [29]:
# Crio um DataFrame para armazenar os Valores agrupados
uf_analises22 = pd.DataFrame({"Numero_de_Transacoes" : uf_operacoes22,
                              "Valor_medio_das_Importacoes" : uf_media22,
                              "Valor_total_das_Importacoes" : uf_total22})

#  Incluo uma coluna no meu DF com passando o 'Index'
uf_analises22["Uf_analises22_id"] = uf_analises22.index

In [30]:
# Edito a formatação numérica dos dados
uf_analises22 = uf_analises22.round(2)
uf_analises22 ["Valor_medio_das_Importacoes"] = uf_analises22["Valor_medio_das_Importacoes"].map("${:,.2f}".format)
uf_analises22 ["Valor_total_das_Importacoes"] = uf_analises22["Valor_total_das_Importacoes"].map("${:,.2f}".format)

In [31]:
# Faço uma 'Reordenação' do DF
uf_analises22 = uf_analises22[['Uf_analises22_id','Numero_de_Transacoes','Valor_total_das_Importacoes',
                               'Valor_medio_das_Importacoes']]

##### Unifico as duas Análises por Estados

In [32]:
# Crio um novo DF 'concatenando' os dois períodos analizados 
uf_analises2122 = pd.concat([uf_analises21,uf_analises22],
                             keys=['Ano_2021','Ano_2022'], axis=1)

In [33]:
# Incluo o 'Index' como propriedade do DF
uf_analises2122["Uf_analises2122_id"] = uf_analises2122.index

# Reordeno a concatenação 
uf_analises2122 = uf_analises2122[['Uf_analises2122_id','Ano_2021','Ano_2022']]

In [34]:
uf_analises2122.sort_values(by="Estado(dest)", ascending=False).head()

Uf_analises2122_id         Ano_2021                       \
                                  Uf_analises21_id Numero_de_Transacoes   
Estado(dest)                                                              
Tocantins               Tocantins        Tocantins                  522   
São Paulo               São Paulo        São Paulo               372536   
Sergipe                   Sergipe          Sergipe                 1568   
Santa Catarina     Santa Catarina   Santa Catarina                80672   
Roraima                   Roraima          Roraima                  223   

                                                                        \
               Valor_total_das_Importacoes Valor_medio_das_Importacoes   
Estado(dest)                                                             
Tocantins                  $246,875,194.00                 $472,940.98   
São Paulo               $32,343,693,350.00                  $86,820.32   
Sergipe                     $68,823,791.00                  $43,892.72   
Santa Catarina          $11,973,722,457.00                 $148,424.76   
Roraima                     $14,569,435.00                  $65,333.79   

                       Ano_2022                       \
               Uf_analises22_id Numero_de_Transacoes   
Estado(dest)                                           
Tocantins             Tocantins                  421   
São Paulo             São Paulo               305648   
Sergipe                 Sergipe                 1123   
Santa Catarina   Santa Catarina                66562   
Roraima                 Roraima                  149   

                                                                        
               Valor_total_das_Importacoes Valor_medio_das_Importacoes  
Estado(dest)                                                            
Tocantins                  $383,427,859.00                 $910,755.01  
São Paulo               $31,028,272,321.00                 $101,516.36  
Sergipe                    $218,532,521.00                 $194,597.08  
Santa Catarina          $11,267,204,084.00                 $169,273.82  
Roraima                     $32,498,423.00                 $218,110.22

## Qual foi o custo das Importações por Municípios?

Detalhamento dos valores movimentados por Municípios no Período.

### Ano 2021

In [35]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
mun_total21 = dadosi21.groupby("Município(dest)")["Total_em_Dólar"].sum()
mun_media21 = dadosi21.groupby("Município(dest)")["Total_em_Dólar"].mean()
mun_operacoes21 = dadosi21.groupby("Município(dest)")["Total_em_Dólar"].count()

In [36]:
# Criando um DataFrame dos Municípios com os valores agrupados
mun_analises21 = pd.DataFrame({"Numero_de_Compras_no_Exterior" : mun_operacoes21,
                               "Valor_medio_das_Importacoes" : mun_media21,
                               "Valor_total_das_Importacoes" : mun_total21})


#  Incluo uma coluna no meu DF com passando o 'Index'
mun_analises21["Mun_analises21_id"] = mun_analises21.index

In [37]:
# Edito a formatação numérica dos dados
mun_analises21 ["Valor_total_das_Importacoes"] = mun_analises21["Valor_total_das_Importacoes"].map("${:,.0f}".format)
mun_analises21 ["Valor_medio_das_Importacoes"] = mun_analises21["Valor_medio_das_Importacoes"].map("${:,.0f}".format)

In [38]:
# Faço uma 'Reordenação' do DF
mun_analises21 = mun_analises21[["Mun_analises21_id","Numero_de_Compras_no_Exterior",
                                 "Valor_total_das_Importacoes", "Valor_medio_das_Importacoes"]]

In [39]:
mun_analises21.sort_values(by="Valor_total_das_Importacoes", ascending=False).head()

,Mun_analises21_id,Numero_de_Compras_no_Exterior,Valor_total_das_Importacoes,Valor_medio_das_Importacoes
Município(dest),,,,
São José dos Campos,São José dos Campos,7231,"$997,376,290","$137,931"
Gramado,Gramado,148,"$995,488","$6,726"
Nova Alvorada,Nova Alvorada,7,"$993,857","$141,980"
Maricá,Maricá,199,"$990,910","$4,979"
Teresina,Teresina,337,"$99,877,965","$296,374"


### Ano 2022

In [40]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
mun_total22 = dadosi22.groupby("Município(dest)")["Total_em_Dólar"].sum()
mun_media22 = dadosi22.groupby("Município(dest)")["Total_em_Dólar"].mean()
mun_operacoes22 = dadosi22.groupby("Município(dest)")["Total_em_Dólar"].count()

In [41]:
# Criando um DataFrame dos Municípios com os valores agrupados
mun_analises22 = pd.DataFrame({"Numero_de_Compras_no_Exterior" : mun_operacoes22,
                               "Valor_medio_das_Importacoes" : mun_media22,
                               "Valor_total_das_Importacoes" : mun_total22})


#  Incluo uma coluna no meu DF com passando o 'Index'
mun_analises22["Mun_analises22_id"] = mun_analises22.index

In [42]:
# Edito a formatação numérica dos dados
mun_analises22 ["Valor_total_das_Importacoes"] = mun_analises22["Valor_total_das_Importacoes"].map("${:,.0f}".format)
mun_analises22 ["Valor_medio_das_Importacoes"] = mun_analises22["Valor_medio_das_Importacoes"].map("${:,.0f}".format)

In [43]:
# Faço uma 'Reordenação' do DF
mun_analises22 = mun_analises22[["Mun_analises22_id","Numero_de_Compras_no_Exterior",
                                 "Valor_total_das_Importacoes", "Valor_medio_das_Importacoes"]]

In [44]:
mun_analises22.sort_values(by="Valor_total_das_Importacoes", ascending=False).head()

,Mun_analises22_id,Numero_de_Compras_no_Exterior,Valor_total_das_Importacoes,Valor_medio_das_Importacoes
Município(dest),,,,
Água Azul do Norte,Água Azul do Norte,6,"$999,273","$166,546"
Ibaiti,Ibaiti,28,"$995,786","$35,564"
São José dos Campos,São José dos Campos,5999,"$991,279,807","$165,241"
Lajes,Lajes,118,"$990,105","$8,391"
Mineiros do Tietê,Mineiros do Tietê,6,"$99,515","$16,586"


## Quais são os principais Remetentes das Importações?

Busco identificar os principais Paises com maiores exportações para o Brasil durante o Período.

### Ano 2021

In [45]:
model_21 = dadosi21["País(orig)"]
model_21.index = range(0, 772039, 1)

In [46]:
model_21

0                  China
1         Estados Unidos
2                  China
3                Polônia
4                  China
               ...      
772034             China
772035           Espanha
772036     Coreia do Sul
772037             China
772038    Estados Unidos
Name: País(orig), Length: 772039, dtype: object

In [47]:
pais_21 = [str(i) for i in model_21]

In [48]:
# Criando um vetor
vetor_pais21 = CountVectorizer(analyzer = 'word', lowercase=False)

In [49]:
# Vetorizando o dataset
paises21 = vetor_pais21.fit_transform(pais_21)

In [50]:
# Retornando paises únicos
paises_unicos21 = vetor_pais21.get_feature_names_out()

In [51]:
# Criar um novo DataFrame dos Paises
dfpaises21 = pd.DataFrame(paises21.todense(), columns = paises_unicos21, index = model_21.index)

In [52]:
# Calcular o percentual 
porcent_pais21 = 100 * pd.Series(dfpaises21.sum()).sort_values(ascending = False) / dfpaises21.shape[0]

In [53]:
# Faço a 'limpeza' de 'Dados' reconhecidos com 'nan'
porcent_pais21.drop(["nan"], inplace=True)

In [54]:
# Crio um DF para melhor aproveitamento dos Dados
porcentagem_pais21 = pd.DataFrame({'Porcentagem21':porcent_pais21})

# Incluo o 'Index' como propriedade do DF
porcentagem_pais21['Pais21_id'] = porcentagem_pais21.index

# Reordeno o DF
porcentagem_pais21 = porcentagem_pais21[['Pais21_id','Porcentagem21']]

In [55]:
porcentagem_pais21.head()

,Pais21_id,Porcentagem21
China,China,16.981137
Unidos,Unidos,11.603041
Estados,Estados,11.536464
Itália,Itália,6.196578
Japão,Japão,4.029589


### Ano 2022

In [56]:
model_22 = dadosi22["País(orig)"]
model_22.index = range(0, 630794, 1)

In [57]:
model_22

0         Tcheca, República
1                  Alemanha
2            Estados Unidos
3                    Itália
4                     Suíça
                ...        
630789                China
630790               Itália
630791               Itália
630792                  NaN
630793                China
Name: País(orig), Length: 630794, dtype: object

In [58]:
pais_22 = [str(i) for i in model_22]

In [59]:
# Criando um vetor
vetor_pais22 = CountVectorizer(analyzer = 'word', lowercase=False)

In [60]:
# Vetorizando o dataset
paises22 = vetor_pais22.fit_transform(pais_22)

In [61]:
# Retornando paises únicos
paises_unicos22 = vetor_pais22.get_feature_names_out()

In [62]:
# Criar um novo DataFrame dos Paises
dfpaises22 = pd.DataFrame(paises22.todense(), columns = paises_unicos22, index = model_22.index)

In [63]:
# Calcular o percentual 
porcent_pais22 = 100 * pd.Series(dfpaises22.sum()).sort_values(ascending = False) / dfpaises22.shape[0]

In [64]:
# Faço a 'limpeza' de 'Dados' reconhecidos com 'nan'
porcent_pais22.drop(["nan"], inplace=True)

In [65]:
# Crio um DF para melhor aproveitamento dos Dados
porcentagem_pais22 = pd.DataFrame({'Porcentagem22':porcent_pais22})

# Incluo o 'Index' como propriedade do DF
porcentagem_pais22['Pais22_id'] = porcentagem_pais22.index

# Reordeno o DF
porcentagem_pais22 = porcentagem_pais22[['Pais22_id','Porcentagem22']]

In [66]:
# Um head dos 5 primeiros 
porcentagem_pais22.head()

,Pais22_id,Porcentagem22
China,China,17.152509
Unidos,Unidos,11.581911
Estados,Estados,11.511048
Itália,Itália,6.153673
Japão,Japão,4.010977


## Quais são os Produtos com Maiores Importações?

Abaixo localizo os produtos mais requisitados por importadoras no Mercado Exterior.

### Ano 2021

In [67]:
# Agrupo os Produtos e realizo a sua Contagem, em seguida, Somo os Valores de todas as suas Compras
produtos_contagem21 = dadosi21.groupby("Produtos")["Total_em_Dólar"].count()
produtos_soma21 = dadosi21.groupby("Produtos")["Total_em_Dólar"].sum()

In [68]:
# Crio um DF com os valores acima
produtosunicos21 = pd.DataFrame({"Quantidade_de_Importacoes" : produtos_contagem21,
                                 "Valor_total_das_Importacoes" : produtos_soma21})

# Incluo o 'Index no meu DF'
produtosunicos21['ProdutosUnicos21_id'] = produtosunicos21.index

In [69]:
# Faço uma pequena alteração na saída 
produtosunicos21["Valor_total_das_Importacoes"] = produtosunicos21["Valor_total_das_Importacoes"].map("${:,.0f}".format)

In [70]:
# Reordeno as 'Colunas' do meu DF
produtosunicos21 = produtosunicos21[['ProdutosUnicos21_id',
                                     'Quantidade_de_Importacoes','Valor_total_das_Importacoes']]

In [71]:
produtosunicos21.head()

,ProdutosUnicos21_id,Quantidade_de_Importacoes,Valor_total_das_Importacoes
Produtos,,,
"Abrasivos naturais ou artificiais, em pó ou em grãos, aplicados sobre matérias têxteis, papel, cartão ou outras matérias, mesmo recortados, costurados ou reunidos de outro modo","Abrasivos naturais ou artificiais, em pó ou em...",1323,"$41,810,364"
"Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria","Absorventes e tampões higiênicos, cueiros e fr...",134,"$20,894,330"
"Acessórios para tubos (por exemplo, uniões, cotovelos, luvas), de alumínio","Acessórios para tubos (por exemplo, uniões, co...",878,"$6,962,756"
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de cobre","Acessórios para tubos [por exemplo: uniões, co...",1820,"$22,653,383"
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de ferro fundido, ferro ou aço","Acessórios para tubos [por exemplo: uniões, co...",5732,"$147,607,324"


### Ano 2022

In [72]:
# Agrupo os Produtos e realizo a sua Contagem, em seguida, Somo os Valores de todas as suas Compras
produtos_contagem22 = dadosi22.groupby("Produtos")["Total_em_Dólar"].count()
produtos_soma22 = dadosi22.groupby("Produtos")["Total_em_Dólar"].sum()

In [73]:
# Crio um DF com os valores acima
produtosunicos22 = pd.DataFrame({"Quantidade_de_Importacoes" : produtos_contagem22,
                                 "Valor_total_das_Importacoes" : produtos_soma22})

# Incluo o 'Index no meu DF'
produtosunicos22['ProdutosUnicos22_id'] = produtosunicos22.index

In [74]:
# Faço uma pequena alteração na saída 
produtosunicos22["Valor_total_das_Importações"] = produtosunicos22["Valor_total_das_Importacoes"].map("${:,.0f}".format)

In [75]:
# Reordeno as 'Colunas' do meu DF
produtosunicos22 = produtosunicos22[['ProdutosUnicos22_id',
                                     'Quantidade_de_Importacoes','Valor_total_das_Importacoes']]

In [76]:
produtosunicos22.head()

,ProdutosUnicos22_id,Quantidade_de_Importacoes,Valor_total_das_Importacoes
Produtos,,,
"Abrasivos naturais ou artificiais, em pó ou em grãos, aplicados sobre matérias têxteis, papel, cartão ou outras matérias, mesmo recortados, costurados ou reunidos de outro modo","Abrasivos naturais ou artificiais, em pó ou em...",1065,37183405
"Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria","Absorventes e tampões higiênicos, cueiros e fr...",90,12780764
"Acessórios para tubos (por exemplo, uniões, cotovelos, luvas), de alumínio","Acessórios para tubos (por exemplo, uniões, co...",746,7392556
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de cobre","Acessórios para tubos [por exemplo: uniões, co...",1559,20296932
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de ferro fundido, ferro ou aço","Acessórios para tubos [por exemplo: uniões, co...",4898,146116375


#### Feito por mim, <a href="https://linkedin.com/in/mario-barcelos"> Mario Barcelos! </a> 